# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [0]:
#Import pandas package
import pandas as pd

#Generate DataFrame from csv file. Setting skip first row and start index on 0
df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

#Change all column names depending on its name. Do it in the same variable
for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

#Get de ids by splitting the index by '('        
names_ids = df.index.str.split('\s\(') 

#The element[0] is the country name, so the new index
df.index = names_ids.str[0]
#The element[1] is the abbreviation or ID (take first 3 characters from that)
df['ID'] = names_ids.str[1].str[:3] 

#Drop ccolumn
df = df.drop('Totals')
df.head()

### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [0]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [0]:
def answer_one():
  #Sort the DataFrame by 'Gold' column and get the first element (the highest)
  ordered = df.sort('Gold', ascending=False)
  ordered['Name'] = ordered.index
  return ordered['Name'].iloc[0]

answer_one()

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [0]:
def answer_two():
  #Get the difference between columns and get the absolute value to short it
  df['Difference'] = (df['Gold'] - df['Gold.1']).abs()
  #Sort the df and get the first element
  ordered = df.sort('Difference',ascending=False)
  ordered['Name'] = ordered.index
  return ordered['Name'].iloc[0]

answer_two()

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [0]:
def answer_three():
  #Get only elements with Gold > 0 and delete others
  filtered = df.where(df['Gold'] > 0)
  filtered = filtered.dropna()
  #Get only elements with Gold.1 > 0 and delete others
  filtered = filtered.where(filtered['Gold.1'] > 0)
  filtered = filtered.dropna()
  #Do the opreation of three columns
  filtered['Difference'] = ((filtered['Gold'] - filtered['Gold.1']).abs())/filtered['Gold.2']
  #Sort the df and get the first element
  ordered = filtered.sort('Difference',ascending=False)
  ordered['Name'] = ordered.index
  return ordered['Name'].iloc[0]

answer_three()

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [0]:
def answer_four():
    Points = []
    values = []
    indexes = []
    #Generate the 2 lists to build the final Series
    for index, row in df.iterrows():
        values.append((row['Gold.2'] * 3) + (row['Silver.2'] * 2) + (row['Bronze.2'] * 1))
        indexes.append(index)

    #Build the Series based on 2 list
    Points = pd.Series(values,index=indexes)
    return Points

answer_four()

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [0]:
census_df = pd.read_csv('census.csv')
census_df.head()

In [0]:
def answer_five():
  #Get only necessary columns
  keep = ['STATE', 'COUNTY', 'STNAME']
  filtered = census_df[keep]
  #Get unique values
  unique_states = census_df['STATE'].unique()
  indexes = []
  values = []
  #Generate lists to build final Series
  for index in unique_states:
    aux = filtered.where(filtered['STATE'] == index)
    aux = aux.dropna()
    indexes.append(aux.iloc[0]['STNAME'])
    values.append(aux.shape[0])

  series = pd.Series(values,index=indexes)   
  #Sort the series and get the first element
  ordered = series.sort_values(ascending=False)
  return ordered.index.values[0]


answer_five()

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [0]:
def answer_six():
  #Get only necessary columns
  keep = ['STNAME', 'CTYNAME', 'SUMLEV', 'CENSUS2010POP']
  kept = census_df[keep]
  
  #Get only data with SUMLEV = 50 and drop others
  filtered = kept[kept['SUMLEV'] == 50]
  filtered = filtered.dropna()
  
  #Sort new dataframe
  filtered = filtered.sort(['STNAME','CENSUS2010POP'],ascending=[1,0])
  #Get unique STANME
  unique_stname = filtered['STNAME'].unique()
  values = []
  indexes = []
  #Genereate 2 list to build final Series
  for index in unique_stname:
    #Get only data STNAME = index and drop others
    counties = filtered.where(filtered['STNAME'] == index)
    counties = counties.dropna()
    max_counties = counties[:3]
    indexes.append(index)
    values.append(max_counties['CENSUS2010POP'].sum())

  #Build the final series and sort it
  series = pd.Series(values,index=indexes) 
  ordered = series.sort_values(ascending=False)
  
  #Get the final results and bild a tuple to return
  final_series = ordered[:3]
  final_indexes = final_series.index
  final_indexes = [final_indexes[0], final_indexes[1], final_indexes[2]]
  return final_indexes

answer_six()

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [0]:
import numpy as np

def answer_seven():
  #Get only necessary columns
  keep = ['STNAME', 'SUMLEV', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 
            'POPESTIMATE2014', 'POPESTIMATE2015']
  kept = census_df[keep]
  #Get only data with SUMLEV = 50 and drop others
  filtered = kept[kept['SUMLEV'] == 50]
  filtered = filtered.dropna()
  
  indexes = []
  values = []
  #Genereate 2 list to build final Series
  for index, row in filtered.iterrows():
    aux = [row['POPESTIMATE2010'],row['POPESTIMATE2011'],row['POPESTIMATE2012'],row['POPESTIMATE2013'],row['POPESTIMATE2014'],row['POPESTIMATE2015']]
    max = np.amax(aux)
    min = np.amin(aux)
    indexes.append(row['CTYNAME'])
    values.append(abs(max-min))

  #Build the final series and sort it
  change_series = pd.Series(values, index=indexes)
  ordered = change_series.sort_values(ascending=False)
  return ordered.index.values[0]

answer_seven()



### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [0]:
def answer_eight():
  #Get only necessary columns
  keep = ['REGION', 'STNAME', 'CTYNAME', 'POPESTIMATE2015', 'POPESTIMATE2014']
  filtered = census_df[keep]
  
  #Get only data with X and drop others
  filtered = filtered.where(filtered[(filtered['REGION'] == 1) | (filtered['REGION'] == 2)])
  filtered = filtered.dropna()
  
  #Get only data with X and drop others
  filtered = filtered.where(filtered['POPESTIMATE2015'] > filtered['POPESTIMATE2014'])
  filtered = filtered.dropna()
  
  #Get only data with X and drop others
  filtered = filtered.where(filtered['CTYNAME'].str.startswith('Washington'))
  filtered = filtered.dropna()
  
  #Get only necessary columns
  keep = ['STNAME', 'CTYNAME']
  filtered = filtered[keep]
  
  #Short df 
  filtered.sort()
  return filtered.sort()

answer_eight()